In [3]:
import os
import os.path as osp
import pandas as pd
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

In [4]:
DATA_DIR = '../../../kgdata'

In [5]:
!ls $DATA_DIR

frame		  sample_submission.csv  validate-sample.zip
frame-sample.zip  validate		 vocabulary.csv


In [4]:
df_sub = pd.read_csv(osp.join(DATA_DIR, 'sample_submission.csv'))
df_sub.head()

,Class,Segments
0,3,002G:35 002G:40 002G:60
1,7,002G:35 002G:40 002G:60
2,8,002G:35 002G:40 002G:60
3,11,002G:35 002G:40 002G:60
4,12,002G:35 002G:40 002G:60


In [5]:
len(df_sub.Class.unique())

1000

In [43]:
0 in df_sub.Class

True

In [12]:
vocab = pd.read_csv(osp.join(DATA_DIR, 'vocabulary.csv'))
vocab.head()

,Index,TrainVideoCount,KnowledgeGraphId,Name,WikiUrl,Vertical1,Vertical2,Vertical3,WikiDescription
0,3,378135,/m/01jddz,Concert,https://en.wikipedia.org/wiki/Concert,Arts & Entertainment,NaN,NaN,A concert is a live music performance in front...
1,7,200813,/m/0k4j,Car,https://en.wikipedia.org/wiki/Car,Autos & Vehicles,NaN,NaN,"A car is a wheeled, self-powered motor vehicle..."
2,8,181579,/m/026bk,Dance,https://en.wikipedia.org/wiki/Dance,Arts & Entertainment,NaN,NaN,Dance is a performance art form consisting of ...
3,11,135357,/m/02wbm,Food,https://en.wikipedia.org/wiki/Food,Food & Drink,NaN,NaN,Food is any substance consumed to provide nutr...
4,12,130835,/m/02vx4,Association football,https://en.wikipedia.org/wiki/Association_foot...,Sports,NaN,NaN,"Association football, more commonly known as f..."


In [13]:
len(vocab.Index.unique())

1000

# video level info

In [6]:
frame_lvl_record = osp.join(DATA_DIR, "frame/train00.tfrecord")

In [15]:
vid_ids = []
labels = []

for example in tf.python_io.tf_record_iterator(frame_lvl_record):
    tf_example = tf.train.Example.FromString(example)
    vid_ids.append(tf_example.features.feature['id']
                   .bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(tf_example.features.feature['labels'].int64_list.value)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [7]:
vid_ids = []
labels = []

for example in tf.python_io.tf_record_iterator(frame_lvl_record):
    tf_example = tf.train.Example.FromString(example)
    vid_ids.append(tf_example.features.feature['id']
                   .bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(tf_example.features.feature['labels'].int64_list.value)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [8]:
print(len(vid_ids), len(labels))

1015 1015


In [9]:
labels[0]

[82, 103, 346, 350]

In [10]:
vid_ids[0]

'op00'

# Frame level data

In [14]:
feat_rgb = []
feat_audio = []

for example in tf.python_io.tf_record_iterator(frame_lvl_record):
    tf_seq_example = tf.train.SequenceExample.FromString(example)
    #print(tf_seq_example)
    #break
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
    print(type(tf_seq_example.feature_lists.feature_list.keys()))
    sess = tf.InteractiveSession()
    rgb_frame = []
    audio_frame = []
    # iterate through frames
    for i in range(n_frames):
        rgb_frame.append(tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['rgb']
                  .feature[i].bytes_list.value[0],tf.uint8)
                       ,tf.float32)).eval())
        audio_frame.append(tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['audio']
                  .feature[i].bytes_list.value[0],tf.uint8)
                       ,tf.float32)).eval())
        
        
    sess.close()
    
    feat_audio.append(audio_frame)
    feat_rgb.append(rgb_frame)
    break

SyntaxError: invalid syntax (<ipython-input-14-8cb1dba0600e>, line 18)

In [19]:
feat_rgb = []
feat_audio = []

for example in tf.python_io.tf_record_iterator(frame_lvl_record):
    tf_seq_example = tf.train.SequenceExample.FromString(example)
    #print(tf_seq_example)
    #break
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
    print(type(tf_seq_example.feature_lists.feature_list.keys()))
    sess = tf.InteractiveSession()
    rgb_frame = []
    audio_frame = []
    # iterate through frames
    for i in range(n_frames):
        rgb_frame.append(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['rgb']
                  .feature[i].bytes_list.value[0],tf.uint8).numpy().astype(np.float32))
        audio_frame.append(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['audio']
                  .feature[i].bytes_list.value[0],tf.uint8).numpy().astype(np.float32))
        
        
    sess.close()
    
    feat_audio.append(audio_frame)
    feat_rgb.append(rgb_frame)
    break

<class 'collections.abc.KeysView'>


In [20]:
feat_rgb[0]

[array([179., 158., 143., ...,  14., 135., 123.], dtype=float32),
 array([201., 141., 141., ...,  49., 134.,  73.], dtype=float32),
 array([138., 169., 115., ...,  35., 215., 103.], dtype=float32),
 array([133., 173., 123., ...,  93., 152., 113.], dtype=float32),
 array([183., 165., 113., ...,  87., 180., 142.], dtype=float32),
 array([169., 112., 128., ..., 223., 255., 219.], dtype=float32),
 array([182., 143., 144., ..., 145., 126., 149.], dtype=float32),
 array([174., 141., 143., ..., 135., 149., 120.], dtype=float32),
 array([156., 161., 129., ..., 143., 181.,  89.], dtype=float32),
 array([146., 162., 120., ...,   0., 156.,  65.], dtype=float32),
 array([226., 143., 149., ...,  48., 223., 105.], dtype=float32),
 array([183., 145., 160., ..., 118., 171.,  99.], dtype=float32),
 array([153., 163., 143., ..., 148.,   9., 184.], dtype=float32),
 array([143., 157., 150., ..., 255.,   0.,  30.], dtype=float32),
 array([192., 135., 173., ..., 117.,   0., 120.], dtype=float32),
 array([15

In [21]:
validate_record = osp.join(DATA_DIR, "validate/validate00.tfrecord")
#validate_record = '/data2/yt8m/3/frame/validate/validate0001.tfrecord'

In [22]:
val_vid_ids = []
val_vid_labels = []
segment_start_times = []
segment_end_times = []
segment_labels = []
segment_scores = []

for example in tf.python_io.tf_record_iterator(validate_record):
    tf_example = tf.train.Example.FromString(example)
    print(tf_example.features.feature)
    #print('1')
    break
    val_vid_ids.append(tf_example.features.feature['id']
                   .bytes_list.value[0].decode(encoding='UTF-8'))
    val_vid_labels.append(tf_example.features.feature['labels'].int64_list.value)
    segment_start_times.append(tf_example.features.feature['segment_start_times'].int64_list.value)
    segment_end_times.append(tf_example.features.feature['segment_end_times'].int64_list.value)
    segment_labels.append(tf_example.features.feature['segment_labels'].int64_list.value)
    segment_scores.append(tf_example.features.feature['segment_scores'].float_list.value)

{'segment_labels': int64_list {
  value: 1036
  value: 1036
  value: 1036
  value: 1036
  value: 1036
}
, 'segment_scores': float_list {
  value: 0.0
  value: 0.0
  value: 1.0
  value: 0.0
  value: 0.0
}
, 'segment_start_times': int64_list {
  value: 145
  value: 110
  value: 135
  value: 155
  value: 70
}
, 'id': bytes_list {
  value: "Iv00"
}
, 'segment_end_times': int64_list {
  value: 150
  value: 115
  value: 140
  value: 160
  value: 75
}
, 'labels': int64_list {
  value: 375
  value: 1036
  value: 1062
}
}


In [36]:
print(val_vid_ids[0])
print(val_vid_labels[0])
print(segment_start_times[0])
print(segment_end_times[0])
print(segment_labels[0])
print(segment_scores[0])

Iv00
[375, 1036, 1062]
[145, 110, 135, 155, 70]
[150, 115, 140, 160, 75]
[1036, 1036, 1036, 1036, 1036]
[0.0, 0.0, 1.0, 0.0, 0.0]


In [37]:
val_vid_ids

['Iv00',
 'ww00',
 'Bv00',
 'pZ00',
 'tA00',
 'Kt00',
 'Y700',
 'Hw00',
 'wR00',
 'nc00',
 'JP00',
 'Vn00',
 '9P00',
 'r400',
 'Qc00',
 'B100']

In [38]:
segment_labels

[[1036, 1036, 1036, 1036, 1036],
 [1031, 1031, 1031, 1031, 1031],
 [666, 666, 666, 666, 666],
 [1753, 1753, 1753, 1753, 1753],
 [465, 475, 465, 465, 475, 475, 475, 465, 475, 465],
 [1672, 1672, 1672, 1672, 1672],
 [833, 833, 833, 833],
 [1400, 1400, 1400, 1400, 1400],
 [1476, 1476, 1476, 1476, 1476],
 [1342, 1342, 1342, 1342, 1342],
 [689, 689, 689, 689, 689],
 [307, 307, 307, 307, 307],
 [462, 462, 462, 462, 462],
 [452, 452, 452, 452, 452],
 [1514, 1514, 1514, 1514, 1514],
 [88, 88, 88, 88, 88]]

In [1]:
segment_scores

NameError: name 'segment_scores' is not defined

In [48]:
np.array(segment_scores)

array([], dtype=float64)

In [9]:
def get_classes_1001():
    df = pd.read_csv('/data2/yt8m/3/frame/classes_1000.csv')
    c = df['class'].values.tolist()
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi
classes, _ = get_classes_1001()

In [11]:
1510 in classes

True